In [53]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import re

# Need these: shop_name,language,year,brand,modell,condition,category_shop,stock_status,stock_text,stock_sizes,url-detail,price,rrp
def get_driver():
    chromeOptions = webdriver.ChromeOptions()

    # Headless is faster. If headless is False then it opens a browser and you can see action of web driver. You can try making it False
    chromeOptions.headless = False
    chromeOptions.add_argument('--log-level=3')

    # installs chrome driver automatically if not present
    s = Service(ChromeDriverManager().install())
    #chromeOptions.add_argument("user-data-dir=/home/bikash/.config/google-chrome/Profile 1")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chromeOptions)
    return driver

In [54]:
driver = get_driver()


In [55]:
rows = []
driver.get('https://www.lucky-bike.de/Fahrraeder/Cube-Aim-SL-2022.html?varselid[1]=de13b84d39a76576fecfd7190dd05715')
import time
wait = WebDriverWait(driver, 10)
wait.until(lambda driver: driver.find_element(By.ID, "usercentrics-root"))
shadow_host = driver.find_element(By.ID, "usercentrics-root")
script = "return arguments[0].shadowRoot"

shadow_root = driver.execute_script(script, shadow_host)
wait.until(
    lambda driver: shadow_root.find_element(By.CLASS_NAME, "sc-eDvSVe.eVGbhR")
)
for i in range(5):
    try:
        shadow_root.find_element(By.CLASS_NAME, "sc-eDvSVe.eVGbhR").click()
        break
    except:
        time.sleep(2)
variants = driver.find_element(By.CLASS_NAME, 'kmt-variantselection-list').find_elements(By.TAG_NAME, 'span')
for i in range(len(variants)):
    driver.find_element(By.CLASS_NAME, 'kmt-variantselection-list').find_elements(By.TAG_NAME, 'span')[i].click()
    stock_sizes = []

    for item in driver.find_element(By.CLASS_NAME, "kmt-select").find_elements(
        By.TAG_NAME, "option"
    )[1:]:
        stock_sizes.append(item.get_attribute("innerText").strip().split("\n")[0])
    status_dict = {}
    count_stock = 0
    no_status = []
    for stock_size in stock_sizes:

        try:
            count_stock += 1
            options = driver.find_element(
                By.CLASS_NAME, "kmt-select"
            ).find_elements(By.TAG_NAME, "option")
            options[count_stock].click()

            wait.until(
                lambda driver: driver.find_element(
                    By.CLASS_NAME, "kmt-shippingselection-status"
                )
            )
            time.sleep(2)
            try:
                status = driver.find_element(
                    By.CLASS_NAME, "kmt-shippingselection-status"
                ).get_attribute("innerText")
            except:
                status = driver.find_element(
                    By.CLASS_NAME, "kmt-shippingselection-delivery"
                ).get_attribute("innerText")

            if status not in status_dict:
                status_dict.update({status: [stock_size]})
            else:
                status_dict[status].append(stock_size)
        except:
            no_status.append(stock_size)
            print(
                "No status for this stock size:",
                stock_size,
                " at "
            )

    stock_sizes = ""
    for status, values in status_dict.items():
        stock_sizes += status + ": " + ", ".join(values) + "\n"
    stock_status=1
    if stock_sizes.strip() == '':
        stock_status=0

    model = driver.find_element(By.CLASS_NAME, "kmt-productmain-title").text.strip()
    year = model.split("|")[0].strip()[-4:]
    color = driver.find_element(By.CLASS_NAME, 'kmt-productmain-variants').find_element(By.TAG_NAME, 'span').text
    print(color)
    if color not in model:
        model+=" | "+color

    stock_text = (
        driver.find_element(By.CLASS_NAME, "kmt-iconlistitem")
        .find_element(By.TAG_NAME, "div")
        .get_attribute("innerText")
    )
    print({"year":year, "stock_sizes": stock_sizes, "stock_text": stock_text, "modell": model, "stock_status":stock_status})

graphite´n´metal
{'year': '2022', 'stock_sizes': 'Sofort lieferbar: 14 Zoll, 16 Zoll, 18 Zoll, 20 Zoll, 22 Zoll, 24 Zoll\n', 'stock_text': 'Lieferzeiten', 'modell': 'Cube Aim SL 2022 | 24 Zoll | graphite´n´metal | 29 Zoll', 'stock_status': 1}
prismagrey´n´blue´n´red
{'year': '2022', 'stock_sizes': 'Sofort lieferbar: 14 Zoll, 16 Zoll, 18 Zoll, 20 Zoll, 22 Zoll, 24 Zoll\n', 'stock_text': 'Lieferzeiten', 'modell': 'Cube Aim SL 2022 | 24 Zoll | prismagrey´n´blue´n´red | 29 Zoll', 'stock_status': 1}


In [56]:
variants

[<selenium.webdriver.remote.webelement.WebElement (session="0a0c8f5264f25b2fb3dcddd4f87979c9", element="725f4b55-3b73-4df9-879e-5a01408f5f77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0a0c8f5264f25b2fb3dcddd4f87979c9", element="7437e6f1-80d8-4b3d-bcc4-b1babc908986")>]